In [1]:
# отключаем предупреждения
import warnings
warnings.filterwarnings("ignore")

# импорт библиотек для анализа
import pandas as pd
import os
import datetime
import numpy as np

# импортируем все библиотеки для работы с pm4py
import pm4py
from pm4py.objects.conversion.log import factory as log_conv
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.filtering.pandas.start_activities import start_activities_filter
from pm4py.algo.filtering.pandas.end_activities import end_activities_filter
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.pandas import case_statistics

# для работы с графами
import graphviz
from graphviz import Digraph

In [2]:
# путь до graphviz executables
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

# Replay

> Подгружаем эталонные логи: тут представлено два процесса, которые мы будем считать эталонными.

In [3]:
etalon = pd.read_excel('\data_custom_replay\etalon.xlsx')

In [4]:
etalon

,case:concept:name,time:timestamp,concept:name
0,urs_1,2018-10-11 23:00:00,action 1
1,urs_1,2018-10-12 23:00:00,action 2
2,urs_1,2018-10-13 23:00:00,action 3
3,urs_1,2018-10-14 23:00:00,action 5
4,usr_2,2018-10-15 23:00:00,action 1
5,usr_2,2018-10-16 23:00:00,action 2
6,usr_2,2018-10-17 23:00:00,action 4
7,usr_2,2018-10-18 23:00:00,action 5


In [5]:
# приведение времени:
etalon['time:timestamp'] = pd.to_datetime(etalon['time:timestamp'])
etalon['time:timestamp'] = etalon['time:timestamp'].astype(np.int64)//10**9
etalon['time:timestamp'] = etalon['time:timestamp'].astype(np.int)

In [6]:
# конвертация в лог и DFG
log_et = log_conv.apply(etalon)
dfg_et = dfg_discovery.apply(log_et)

In [7]:
# смотрим вариации представленные в эталонном логе
variants_et = variants_filter.get_variants(log_et)

In [8]:
pd.DataFrame(variants_et).columns

Index(['action 1,action 2,action 3,action 5', 'action 1,action 2,action 4,action 5'], dtype='object')

> Теперь подгружаем синтезированные реальные логи

In [9]:
dfr=pd.read_excel('\data_custom_replay\logs.xlsx')

In [10]:
dfr.head()

,case:concept:name,time:timestamp,concept:name
0,urs_3,2018-10-11 23:00:00,action 1
1,urs_3,2018-10-12 23:00:00,action 2
2,urs_3,2018-10-13 23:00:00,action 4
3,urs_4,2018-10-14 23:00:00,action 1
4,urs_4,2018-10-15 23:00:00,action 2


In [11]:
log_block = log_conv.apply(dfr)

In [12]:
# варианты прохождений реальных логов
variants_count_logs = case_statistics.get_variant_statistics(dfr, parameters={case_statistics.Parameters.CASE_ID_KEY: "case:concept:name",
                                                      case_statistics.Parameters.ACTIVITY_KEY: "concept:name",
                                                      case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})
variants_count_logs = sorted(variants_count_logs, key=lambda x: x['case:concept:name'], reverse=True)

In [13]:
variants_count_logs

[{'variant': 'action 1,action 2,action 5', 'case:concept:name': 1},
 {'variant': 'action 1,action 2,action 4,action 5', 'case:concept:name': 1},
 {'variant': 'action 1,action 2,action 4', 'case:concept:name': 1},
 {'variant': 'action 1,action 2,action 3,action 5', 'case:concept:name': 1},
 {'variant': 'action 1,action 2,action 3', 'case:concept:name': 1}]

In [14]:
# варианты traces эталонного процесса
variants_count_e = case_statistics.get_variant_statistics(etalon, parameters={case_statistics.Parameters.CASE_ID_KEY: "case:concept:name",
                                                      case_statistics.Parameters.ACTIVITY_KEY: "concept:name",
                                                      case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})
variants_count_e = sorted(variants_count_e, key=lambda x: x['case:concept:name'], reverse=True)

In [15]:
variants_count_e

[{'variant': 'action 1,action 2,action 4,action 5', 'case:concept:name': 1},
 {'variant': 'action 1,action 2,action 3,action 5', 'case:concept:name': 1}]

In [16]:
# приведение к формату таблиц
et_var = pd.DataFrame(variants_count_e)
et_var['variant']=et_var['variant'].map(lambda x: x.split(','))

variants_logs = pd.DataFrame(variants_count_logs)
variants_logs['variant'] = variants_logs['variant'].map(lambda x: x.split(','))

In [17]:
rea = variants_logs['variant'].to_list()
ide = et_var['variant'].to_list()

In [18]:
"""
функция поиска пропущенных токенов, на вход подаются параметры
:params: etalon_list - список дейтсвий в эталонном процесса
:params: real_list - список действий в реальном логе
возвращается список пропущенных токенов
"""
def missing_tokens(etalon_list, real_list):
    missing = []
    for i in range(len(etalon_list)):
        if etalon_list[i]!=real_list[i]:
            missing.append(etalon_list[i])
        else:
            pass
    return missing

In [19]:
"""
функция поиска расхождений, на вход необходимо подать следующие параметры
если длина лога отличается от ближайшего эталонного процесса, дополняем нуляеми разницу в длинах
:params: real - список реальных traces из логов
:params: ideal - список traces из эталонного лога
функция возвращает списки расхождений и новые (дополненные нулями) исходные логи
"""
def preparation_logs(real, ideal):
    global_ = []
    for l in range(len(real)):
        loc = []
        for k in range(len(ideal)):
            if len(real[l])==len(ideal[k]):
                loc.append(missing_tokens(ideal[k], real[l]))
            elif len(real[l])<len(ideal[k]):
                dif_len = len(ideal[k])-len(real[l])
                for add in range(0,dif_len):
                    real[l].append(add)
                loc.append(missing_tokens(ideal[k], real[l]))
            elif len(real[l])>len(ideal[k]):
                m_loc = []
                for t in range(len(ideal[k])):
                    micro_loc = []
                    if ideal[k][t]!=real[l][t]:
                        micro_loc.append(ideal[k][t])
                    if len(micro_loc)>0:
                        m_loc.append(micro_loc)
                loc.append(m_loc)
        global_.append(loc)
    return global_, real, ideal 

In [20]:
globals_, reals, ideals = preparation_logs(rea, ide)

In [21]:
len_list =[]
for process in range(len(globals_)):
    len_list.append(min(map(len,globals_[process])))

In [22]:
def final_deviatinos(global_):
    final_ = []
    for i in range(len(global_)):
        loc_final_list=[]
        for j in range(len(global_[i])):
            if len(global_[i][j])==len_list[i]:
                loc_final_list.append(global_[i][j])
        final_.append(loc_final_list)
    return final_

In [23]:
finals = final_deviatinos(globals_)

In [24]:
variants_logs['missing_tokens'] = finals

In [25]:
variants_logs['copy_tokens'] = variants_logs['missing_tokens'].map(lambda x: set([j for i in x for j in i]))

In [26]:
all_nodes = list(set(dfr['concept:name']))

In [29]:
# not declared = missing tokens
nd = dict.fromkeys(all_nodes)
for i in all_nodes:
    t = 0
    for line in range(len(variants_logs)):
        if i in str(variants_logs['copy_tokens'].iloc[line]):
            t+=int(variants_logs['case:concept:name'].iloc[line])
    nd[i] =t

In [30]:
dfg_replay = dfg_discovery.apply(log_et)

In [31]:
# ищем события начала
log_start_r = start_activities_filter.get_start_activities(etalon)

In [32]:
# ищем события конца
end_acitivites_r = end_activities_filter.get_end_activities(etalon)

In [33]:
concepts_and_numbers = dfr.groupby(['concept:name'])[['case:concept:name']].count()

In [34]:
count_cases = variants_logs['case:concept:name'].sum()

In [36]:
# составляем схему связей между нодами
schema=pd.DataFrame(columns = ['node1', 'node2'])
nodes1=[]
nodes2 =[]
for t in set(etalon['concept:name'].to_list()):
    for j in dfg_replay:
        if t in j[0]:
            print(t,j)
            nodes1.append(j[0])
            nodes2.append(j[1])
schema['node1']=nodes1
schema['node2']=nodes2

action 1 ('action 1', 'action 2')
action 3 ('action 3', 'action 5')
action 2 ('action 2', 'action 3')
action 2 ('action 2', 'action 4')
action 4 ('action 4', 'action 5')


In [37]:
schema

,node1,node2
0,action 1,action 2
1,action 3,action 5
2,action 2,action 3
3,action 2,action 4
4,action 4,action 5


In [38]:
or_counter = 0
g = Digraph('G', filename = 'replay_DFG.gv')
# добавление точки входа
g.node('SP', color = 'green')
g.node('EP', color = 'orange')

for i in set(log_start_r.keys()):
    g.attr('node',shape = 'box' )
    g.node(str(i),label = str(i)+' (' +str(nd[i])+',' +str(count_cases)+')')
    g.edge('SP', i)

# теперь добавляются все ноды, которые участвуют в процессе, помимо использованных, как начальные точки
for i in set(etalon['concept:name'])-set(log_start_r.keys()):
    g.attr('node', shape = 'box' )
    g.node(str(i),label = str(i)+' (' +str(nd[i])+','+str(count_cases)+')')


for i in range(len(schema)):
    g.edge(schema['node1'].iloc[i],schema['node2'].iloc[i])
    
# связывание точки выхода и конечных событий
for i in set(end_acitivites_r.keys()):
    g.edge(i,'EP')

In [40]:
# сохранение dot - файла 
g.render(filename='replay.dot')

'replay.dot.pdf'

In [41]:
# автопросмотр
g.view()

'replay.dot.pdf'